In [1]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [4]:
AppleR = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Apple/Apple_Ferdig.csv")

In [5]:
AppleR.shape

(10189, 16)

In [6]:
AppleR.head(3)

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,author_link,article_content,appears_in,symbols,related_articles,Time.1
0,11,2018-03-13,18:30:00,11,474,{'$oid': '5ab8d2273efc2c03c520d021'},https://www.nasdaq.com/article/with-38-million...,"With 38 Million Subscribers, Apple Music Growt...",{'$date': '2018-03-12T18:30:00Z'},"By Evan Niu, CFA, Motley Fool",https://www.nasdaq.com/author/fool,It's been just over a month since Apple(NASDAQ...,"PersonalFinance,Stocks",AAPL,NaN,{'$date': '2018-03-12T18:30:00Z'}
1,12,2018-03-13,18:05:00,12,475,{'$oid': '5ab8d22f3efc2c03c520d033'},https://www.nasdaq.com/article/report-apple-in...,Report: Apple Inc. Prepping a New 13.3-inch Ma...,{'$date': '2018-03-12T18:05:00Z'},"By Ashraf Eassa, Motley Fool",https://www.nasdaq.com/author/fool,A new report from DIGITIMES asserts that Apple...,"PersonalFinance,Stocks",AAPL,NaN,{'$date': '2018-03-12T18:05:00Z'}
2,16,2018-03-14,17:45:00,16,777,{'$oid': '5ab8d5993efc2c03c520d7f1'},https://www.nasdaq.com/article/could-intel-cor...,Could Intel Corporation Actually Buy Broadcom ...,{'$date': '2018-03-13T17:45:00Z'},"By Leo Sun, Motley Fool",https://www.nasdaq.com/author/fool,Intel(NASDAQ: INTC) reportedly considered buy...,"PersonalFinance,Stocks","AVGO,INTC,QCOM,AAPL",NaN,{'$date': '2018-03-13T17:45:00Z'}


In [7]:
AppleAksje = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Apple/AppleStockPrices_Label.csv")

In [8]:
AppleAksje.shape

(10274, 9)

In [9]:
AppleAksje.head(3)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Label
0,1980-12-12,0.100600,0.101037,0.100600,0.100600,469033600,0.0,0.0,0
1,1980-12-15,0.095789,0.095789,0.095352,0.095352,175884800,0.0,0.0,0
2,1980-12-16,0.088790,0.088790,0.088353,0.088353,105728000,0.0,0.0,0


In [10]:
#Merge datasettene
merge = AppleR.merge(AppleAksje, how = "inner", on = "Date")

In [11]:
#Create a function to get the subjecticity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a fuction to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [12]:
#Create two new columns "Subjectivity" and "Polarity"
merge["Subjectivity"] = merge["article_content"].apply(getSubjectivity)
merge["Polarity"] = merge["article_content"].apply(getPolarity)

In [13]:
import nltk

from nltk.corpus import stopwords
stop = stopwords.words('english')

In [14]:
merge['article_content_without_stopwords'] = merge['article_content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [15]:
from nltk.stem import PorterStemmer

port = PorterStemmer()

In [16]:
def stem_words(article_content_without_stopwords):
    return " ".join([port.stem(word) for word in article_content_without_stopwords.split()])

merge["EtterStemming"] = merge["article_content_without_stopwords"].apply(lambda article_content_without_stopwords: stem_words(article_content_without_stopwords))
merge.head()

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,...,Low,Close,Volume,Dividends,Stock Splits,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming
0,11,2018-03-13,18:30:00,11,474,{'$oid': '5ab8d2273efc2c03c520d021'},https://www.nasdaq.com/article/with-38-million...,"With 38 Million Subscribers, Apple Music Growt...",{'$date': '2018-03-12T18:30:00Z'},"By Evan Niu, CFA, Motley Fool",...,43.094648,43.270161,126774000,0.0,0.0,0,0.535762,0.259726,It's month since Apple(NASDAQ: AAPL) reportedl...,it' month sinc apple(nasdaq: aapl) reportedli ...
1,12,2018-03-13,18:05:00,12,475,{'$oid': '5ab8d22f3efc2c03c520d033'},https://www.nasdaq.com/article/report-apple-in...,Report: Apple Inc. Prepping a New 13.3-inch Ma...,{'$date': '2018-03-12T18:05:00Z'},"By Ashraf Eassa, Motley Fool",...,43.094648,43.270161,126774000,0.0,0.0,0,0.557425,0.166736,A new report DIGITIMES asserts Apple(NASDAQ: A...,a new report digitim assert apple(nasdaq: aapl...
2,37,2018-03-13,16:50:00,37,1651,{'$oid': '5ab8d26e3efc2c03c520d0b7'},https://www.nasdaq.com/article/apple-roundup-i...,"Apple Roundup: iPhone X, Irish Tax, New Headph...",{'$date': '2018-03-12T16:50:00Z'},"By Sejuti Banerjea, Zacks.com",...,43.094648,43.270161,126774000,0.0,0.0,0,0.385577,0.034884,Analyst reports continue indicate iPhone X dem...,analyst report continu indic iphon x demand la...
3,38,2018-03-13,16:12:00,38,1660,{'$oid': '5ab8d2b83efc2c03c520d159'},https://www.nasdaq.com/article/3-strong-buy-se...,3 Strong Buy Semiconductor Stocks to Consider Now,{'$date': '2018-03-12T16:12:00Z'},"By Ryan McQueeney, Zacks.com",...,43.094648,43.270161,126774000,0.0,0.0,0,0.489353,0.146362,Semiconductor stocks battered recent market se...,semiconductor stock batter recent market sell-...
4,8,2018-03-13,07:00:00,8,356,{'$oid': '5ab8d0723efc2c03c520cc4f'},https://www.nasdaq.com/article/apple-google-se...,"Apple, Google see reputation of corporate bran...",{'$date': '2018-03-13T07:00:00Z'},By Reuters,...,43.094648,43.270161,126774000,0.0,0.0,0,0.420026,0.163776,By Stephen NellisMarch 13 (Reuters) - Apple In...,by stephen nellismarch 13 (reuters) - appl inc...


In [17]:
from nltk.text import TextCollection

def vectorize(corpus):
    corpus = [tokenize(EtterStemming) for EtterStemming in corpus]
    texts = TextCollection(corpus)
    
    for EtterStemming in corpus:
        yield {
            term: texts.tf_idf(term, EtterStemming)
            for term in EtterStemming
        }

In [18]:
merge["TF-IDF"] = merge["EtterStemming"].apply(lambda EtterStemming: vectorize(EtterStemming))
merge.head()

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,...,Close,Volume,Dividends,Stock Splits,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming,TF-IDF
0,11,2018-03-13,18:30:00,11,474,{'$oid': '5ab8d2273efc2c03c520d021'},https://www.nasdaq.com/article/with-38-million...,"With 38 Million Subscribers, Apple Music Growt...",{'$date': '2018-03-12T18:30:00Z'},"By Evan Niu, CFA, Motley Fool",...,43.270161,126774000,0.0,0.0,0,0.535762,0.259726,It's month since Apple(NASDAQ: AAPL) reportedl...,it' month sinc apple(nasdaq: aapl) reportedli ...,<generator object vectorize at 0x7f7e58792a50>
1,12,2018-03-13,18:05:00,12,475,{'$oid': '5ab8d22f3efc2c03c520d033'},https://www.nasdaq.com/article/report-apple-in...,Report: Apple Inc. Prepping a New 13.3-inch Ma...,{'$date': '2018-03-12T18:05:00Z'},"By Ashraf Eassa, Motley Fool",...,43.270161,126774000,0.0,0.0,0,0.557425,0.166736,A new report DIGITIMES asserts Apple(NASDAQ: A...,a new report digitim assert apple(nasdaq: aapl...,<generator object vectorize at 0x7f7e588d8900>
2,37,2018-03-13,16:50:00,37,1651,{'$oid': '5ab8d26e3efc2c03c520d0b7'},https://www.nasdaq.com/article/apple-roundup-i...,"Apple Roundup: iPhone X, Irish Tax, New Headph...",{'$date': '2018-03-12T16:50:00Z'},"By Sejuti Banerjea, Zacks.com",...,43.270161,126774000,0.0,0.0,0,0.385577,0.034884,Analyst reports continue indicate iPhone X dem...,analyst report continu indic iphon x demand la...,<generator object vectorize at 0x7f7e588d8f90>
3,38,2018-03-13,16:12:00,38,1660,{'$oid': '5ab8d2b83efc2c03c520d159'},https://www.nasdaq.com/article/3-strong-buy-se...,3 Strong Buy Semiconductor Stocks to Consider Now,{'$date': '2018-03-12T16:12:00Z'},"By Ryan McQueeney, Zacks.com",...,43.270161,126774000,0.0,0.0,0,0.489353,0.146362,Semiconductor stocks battered recent market se...,semiconductor stock batter recent market sell-...,<generator object vectorize at 0x7f7e588d87b0>
4,8,2018-03-13,07:00:00,8,356,{'$oid': '5ab8d0723efc2c03c520cc4f'},https://www.nasdaq.com/article/apple-google-se...,"Apple, Google see reputation of corporate bran...",{'$date': '2018-03-13T07:00:00Z'},By Reuters,...,43.270161,126774000,0.0,0.0,0,0.420026,0.163776,By Stephen NellisMarch 13 (Reuters) - Apple In...,by stephen nellismarch 13 (reuters) - appl inc...,<generator object vectorize at 0x7f7e588d89e0>


In [19]:
merge.head()

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,...,Close,Volume,Dividends,Stock Splits,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming,TF-IDF
0,11,2018-03-13,18:30:00,11,474,{'$oid': '5ab8d2273efc2c03c520d021'},https://www.nasdaq.com/article/with-38-million...,"With 38 Million Subscribers, Apple Music Growt...",{'$date': '2018-03-12T18:30:00Z'},"By Evan Niu, CFA, Motley Fool",...,43.270161,126774000,0.0,0.0,0,0.535762,0.259726,It's month since Apple(NASDAQ: AAPL) reportedl...,it' month sinc apple(nasdaq: aapl) reportedli ...,<generator object vectorize at 0x7f7e58792a50>
1,12,2018-03-13,18:05:00,12,475,{'$oid': '5ab8d22f3efc2c03c520d033'},https://www.nasdaq.com/article/report-apple-in...,Report: Apple Inc. Prepping a New 13.3-inch Ma...,{'$date': '2018-03-12T18:05:00Z'},"By Ashraf Eassa, Motley Fool",...,43.270161,126774000,0.0,0.0,0,0.557425,0.166736,A new report DIGITIMES asserts Apple(NASDAQ: A...,a new report digitim assert apple(nasdaq: aapl...,<generator object vectorize at 0x7f7e588d8900>
2,37,2018-03-13,16:50:00,37,1651,{'$oid': '5ab8d26e3efc2c03c520d0b7'},https://www.nasdaq.com/article/apple-roundup-i...,"Apple Roundup: iPhone X, Irish Tax, New Headph...",{'$date': '2018-03-12T16:50:00Z'},"By Sejuti Banerjea, Zacks.com",...,43.270161,126774000,0.0,0.0,0,0.385577,0.034884,Analyst reports continue indicate iPhone X dem...,analyst report continu indic iphon x demand la...,<generator object vectorize at 0x7f7e588d8f90>
3,38,2018-03-13,16:12:00,38,1660,{'$oid': '5ab8d2b83efc2c03c520d159'},https://www.nasdaq.com/article/3-strong-buy-se...,3 Strong Buy Semiconductor Stocks to Consider Now,{'$date': '2018-03-12T16:12:00Z'},"By Ryan McQueeney, Zacks.com",...,43.270161,126774000,0.0,0.0,0,0.489353,0.146362,Semiconductor stocks battered recent market se...,semiconductor stock batter recent market sell-...,<generator object vectorize at 0x7f7e588d87b0>
4,8,2018-03-13,07:00:00,8,356,{'$oid': '5ab8d0723efc2c03c520cc4f'},https://www.nasdaq.com/article/apple-google-se...,"Apple, Google see reputation of corporate bran...",{'$date': '2018-03-13T07:00:00Z'},By Reuters,...,43.270161,126774000,0.0,0.0,0,0.420026,0.163776,By Stephen NellisMarch 13 (Reuters) - Apple In...,by stephen nellismarch 13 (reuters) - appl inc...,<generator object vectorize at 0x7f7e588d89e0>


In [23]:
merge.to_csv("AppleTFIDF.csv")

In [20]:
#Create a function to get the sentiment scores
def getSIA(EtterStemming):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(EtterStemming)
    return sentiment

In [21]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/joakimlarsen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [22]:
#Get the sentiment scores for each day
compound = []
neg = []
pos = []
neu = []
SIA = 0

for i in range(0, len(merge["EtterStemming"])):
    SIA = getSIA(merge["EtterStemming"][i])
    compound.append(SIA["compound"])
    neg.append(SIA["neg"])
    neu.append(SIA["neu"])
    pos.append(SIA["pos"])

In [24]:
#Store the sentiment scores in the merge data set:
merge["Compound"] = compound
merge["Negative"] = neg
merge["Neutral"] = neu
merge["Positive"] = pos

In [25]:
#Show the merge data
merge.head(3)

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,...,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming,TF-IDF,Compound,Negative,Neutral,Positive
0,11,2018-03-13,18:30:00,11,474,{'$oid': '5ab8d2273efc2c03c520d021'},https://www.nasdaq.com/article/with-38-million...,"With 38 Million Subscribers, Apple Music Growt...",{'$date': '2018-03-12T18:30:00Z'},"By Evan Niu, CFA, Motley Fool",...,0,0.535762,0.259726,It's month since Apple(NASDAQ: AAPL) reportedl...,it' month sinc apple(nasdaq: aapl) reportedli ...,<generator object vectorize at 0x7f7e58792a50>,0.9932,0.052,0.788,0.160
1,12,2018-03-13,18:05:00,12,475,{'$oid': '5ab8d22f3efc2c03c520d033'},https://www.nasdaq.com/article/report-apple-in...,Report: Apple Inc. Prepping a New 13.3-inch Ma...,{'$date': '2018-03-12T18:05:00Z'},"By Ashraf Eassa, Motley Fool",...,0,0.557425,0.166736,A new report DIGITIMES asserts Apple(NASDAQ: A...,a new report digitim assert apple(nasdaq: aapl...,<generator object vectorize at 0x7f7e588d8900>,0.9888,0.046,0.815,0.139
2,37,2018-03-13,16:50:00,37,1651,{'$oid': '5ab8d26e3efc2c03c520d0b7'},https://www.nasdaq.com/article/apple-roundup-i...,"Apple Roundup: iPhone X, Irish Tax, New Headph...",{'$date': '2018-03-12T16:50:00Z'},"By Sejuti Banerjea, Zacks.com",...,0,0.385577,0.034884,Analyst reports continue indicate iPhone X dem...,analyst report continu indic iphon x demand la...,<generator object vectorize at 0x7f7e588d8f90>,0.9940,0.054,0.838,0.109


In [26]:
#Create a list of column to keep
keep_columns = ["Open", "Low", "High", "Date", "Close", "Volume", "Subjectivity", "Polarity", "Compound", "Negative", "Neutral", "Positive", "Label"]
df = merge[keep_columns]
df

,Open,Low,High,Date,Close,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,43.900086,43.094648,44.118878,2018-03-13,43.270161,126774000,0.535762,0.259726,0.9932,0.052,0.788,0.160,0
1,43.900086,43.094648,44.118878,2018-03-13,43.270161,126774000,0.557425,0.166736,0.9888,0.046,0.815,0.139,0
2,43.900086,43.094648,44.118878,2018-03-13,43.270161,126774000,0.385577,0.034884,0.9940,0.054,0.838,0.109,0
3,43.900086,43.094648,44.118878,2018-03-13,43.270161,126774000,0.489353,0.146362,0.9948,0.016,0.844,0.140,0
4,43.900086,43.094648,44.118878,2018-03-13,43.270161,126774000,0.420026,0.163776,-0.0772,0.097,0.810,0.093,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8618,50.633720,50.436950,50.960850,2019-08-26,50.788677,104174400,0.498261,0.176083,0.9979,0.034,0.808,0.159,1
8619,50.633720,50.436950,50.960850,2019-08-26,50.788677,104174400,0.546523,-0.052710,-0.0772,0.114,0.777,0.110,1
8620,50.633720,50.436950,50.960850,2019-08-26,50.788677,104174400,0.456937,0.192916,0.9999,0.051,0.725,0.224,1
8621,50.633720,50.436950,50.960850,2019-08-26,50.788677,104174400,0.496731,0.077628,0.7579,0.070,0.823,0.107,1


In [27]:
df.to_csv("AppleFørTrening.csv")

In [27]:
AppleTest = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Apple/TestAppleUnik.csv")

In [28]:
AppleTest.shape

(243, 14)

In [29]:
AppleTest.head(3)

,X,Open,Low,High,Date,Close,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,6531,54.071930,53.866055,55.285368,2018-08-30,54.503052,195175200,0.442600,0.154945,0.9403,0.034,0.873,0.094,1
1,6596,54.861504,54.737982,55.433104,2018-08-31,55.132774,173360400,0.277222,0.079217,0.5106,0.056,0.877,0.066,1
2,8545,55.321696,54.890575,55.508190,2018-09-04,55.309586,109560400,0.500270,0.190458,0.9844,0.043,0.849,0.108,0


In [30]:
AppleTrain = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Apple/TrainAppleUnik.csv")

In [31]:
AppleTrain.shape

(1553, 14)

In [32]:
AppleTrain.head(3)

,X,Open,Low,High,Date,Close,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,8502,5.895322,5.833379,5.897468,2009-10-14,5.865884,375510800,0.393429,0.126336,0.9718,0.062,0.831,0.107,0
1,6504,6.201664,6.035767,6.217917,2009-12-01,6.040061,465763200,0.483192,0.087219,0.9985,0.056,0.767,0.176,0
2,6660,5.928134,5.785848,5.941933,2009-12-07,5.794127,714758800,0.499747,0.126303,0.9979,0.056,0.831,0.112,0


In [33]:
#Create a list of column to keep
keep_columns1 = ["Open", "Low", "High", "Volume","Close", "Subjectivity", "Polarity", "Compound", "Negative", "Neutral", "Positive", "Label"]
df1 = AppleTest[keep_columns1]
df1

,Open,Low,High,Volume,Close,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,54.071930,53.866055,55.285368,195175200,54.503052,0.442600,0.154945,0.9403,0.034,0.873,0.094,1
1,54.861504,54.737982,55.433104,173360400,55.132774,0.277222,0.079217,0.5106,0.056,0.877,0.066,1
2,55.321696,54.890575,55.508190,109560400,55.309586,0.500270,0.190458,0.9844,0.043,0.849,0.108,0
3,55.462178,54.520006,55.626874,133332000,54.948704,0.433944,0.039827,0.9940,0.014,0.848,0.138,0
4,54.793689,53.599628,55.064960,137160000,54.035595,0.410280,0.135265,0.9840,0.035,0.856,0.109,0
...,...,...,...,...,...,...,...,...,...,...,...,...
238,52.436624,51.836476,52.744076,89014800,52.257072,0.450048,0.184692,0.9945,0.055,0.794,0.151,0
239,51.511804,49.438347,52.156227,187272000,49.841724,0.359338,0.062356,0.2350,0.056,0.881,0.063,0
240,50.633720,50.436950,50.960850,104174400,50.788677,0.498261,0.176083,0.9979,0.034,0.808,0.159,1
241,51.125642,50.060627,51.295356,103493200,50.215584,0.433536,0.170572,0.9349,0.046,0.850,0.104,0


In [34]:
#Create the feature data set
X = df1
X = np.array(X.drop(["Label"], 1))

#Create the target data set
y = np.array(df1["Label"])

/var/folders/m7/01kjn8yx6d3_tjkcr1dkd9g80000gn/T/ipykernel_12494/3997290794.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(X.drop(["Label"], 1))


In [35]:
X

array([[5.40719298e+01, 5.38660554e+01, 5.52853680e+01, ...,
        3.40000000e-02, 8.73000000e-01, 9.40000000e-02],
       [5.48615040e+01, 5.47379815e+01, 5.54331043e+01, ...,
        5.60000000e-02, 8.77000000e-01, 6.60000000e-02],
       [5.53216965e+01, 5.48905746e+01, 5.55081905e+01, ...,
        4.30000000e-02, 8.49000000e-01, 1.08000000e-01],
       ...,
       [5.06337200e+01, 5.04369497e+01, 5.09608498e+01, ...,
        3.40000000e-02, 8.08000000e-01, 1.59000000e-01],
       [5.11256421e+01, 5.00606266e+01, 5.12953564e+01, ...,
        4.60000000e-02, 8.50000000e-01, 1.04000000e-01],
       [5.02008275e+01, 5.00089775e+01, 5.05992846e+01, ...,
        6.20000000e-02, 8.15000000e-01, 1.23000000e-01]])

In [36]:
y

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1])

In [37]:
x_test = X

In [38]:
len(x_test)

243

In [39]:
y_test = y

In [40]:
len(y_test)

243

In [41]:
#Create a list of column to keep
keep_columns2 = ["Open", "Low", "High", "Volume", "Close", "Subjectivity", "Polarity", "Compound", "Negative", "Neutral", "Positive", "Label"]
df2 = AppleTrain[keep_columns2]
df2

,Open,Low,High,Volume,Close,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,5.895322,5.833379,5.897468,375510800,5.865884,0.393429,0.126336,0.9718,0.062,0.831,0.107,0
1,6.201664,6.035767,6.217917,465763200,6.040061,0.483192,0.087219,0.9985,0.056,0.767,0.176,0
2,5.928134,5.785848,5.941933,714758800,5.794127,0.499747,0.126303,0.9979,0.056,0.831,0.112,0
3,5.865577,5.835831,6.076551,684782000,6.065512,0.539286,0.167857,0.9097,0.040,0.835,0.125,1
4,6.241834,6.235701,6.419691,500889200,6.410184,0.432539,0.117698,0.8934,0.025,0.913,0.062,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1548,51.988967,51.976860,52.570258,75532800,52.192421,0.508349,0.140143,0.9803,0.063,0.817,0.120,1
1549,52.461276,52.100391,52.533934,73905600,52.354706,0.499142,0.111774,0.9599,0.029,0.892,0.079,0
1550,52.594489,52.395884,52.979595,82100400,52.785831,0.446054,0.125820,0.9867,0.051,0.841,0.108,1
1551,53.044976,53.023179,53.415547,91107200,53.212097,0.357170,0.080684,0.9698,0.022,0.906,0.072,1


In [42]:
#Create the feature data set
X1 = df2
X1 = np.array(X1.drop(["Label"], 1))

#Create the target data set
y1 = np.array(df2["Label"])

/var/folders/m7/01kjn8yx6d3_tjkcr1dkd9g80000gn/T/ipykernel_12494/791745654.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X1 = np.array(X1.drop(["Label"], 1))


In [43]:
X1

array([[5.89532155e+00, 5.83337914e+00, 5.89746773e+00, ...,
        6.20000000e-02, 8.31000000e-01, 1.07000000e-01],
       [6.20166401e+00, 6.03576734e+00, 6.21791677e+00, ...,
        5.60000000e-02, 7.67000000e-01, 1.76000000e-01],
       [5.92813357e+00, 5.78584817e+00, 5.94193267e+00, ...,
        5.60000000e-02, 8.31000000e-01, 1.12000000e-01],
       ...,
       [5.25944886e+01, 5.23958837e+01, 5.29795950e+01, ...,
        5.10000000e-02, 8.41000000e-01, 1.08000000e-01],
       [5.30449762e+01, 5.30231788e+01, 5.34155472e+01, ...,
        2.20000000e-02, 9.06000000e-01, 7.20000000e-02],
       [5.33210907e+01, 5.31418625e+01, 5.41300531e+01, ...,
        1.90000000e-02, 8.58000000e-01, 1.23000000e-01]])

In [44]:
y1

array([0, 0, 0, ..., 1, 1, 1])

In [45]:
x_train = X1

In [46]:
len(X1)

1553

In [47]:
y_train = y1

In [48]:
len(y1)

1553

In [49]:
#Create and train the model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [50]:
#Show the models predictions
predictions = model.predict(x_test)
predictions

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1])

In [51]:
y_test

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1])

In [52]:
#Show the model metrics
print( classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       120
           1       1.00      1.00      1.00       123

    accuracy                           1.00       243
   macro avg       1.00      1.00      1.00       243
weighted avg       1.00      1.00      1.00       243



In [53]:
#Create and train the model (Naive Bayes)
from sklearn.naive_bayes import GaussianNB
modelNB = GaussianNB()

In [54]:
#Train the model
modelNB.fit(x_train, y_train)

GaussianNB()

In [55]:
#Accuracy for modellen
modelNB.score(x_test, y_test)

0.5061728395061729

In [56]:
x_test[:10]

array([[ 5.40719298e+01,  5.38660554e+01,  5.52853680e+01,
         1.95175200e+08,  5.45030518e+01,  4.42600000e-01,
         1.54945455e-01,  9.40300000e-01,  3.40000000e-02,
         8.73000000e-01,  9.40000000e-02],
       [ 5.48615040e+01,  5.47379815e+01,  5.54331043e+01,
         1.73360400e+08,  5.51327744e+01,  2.77222222e-01,
         7.92171717e-02,  5.10600000e-01,  5.60000000e-02,
         8.77000000e-01,  6.60000000e-02],
       [ 5.53216965e+01,  5.48905746e+01,  5.55081905e+01,
         1.09560400e+08,  5.53095856e+01,  5.00270147e-01,
         1.90457875e-01,  9.84400000e-01,  4.30000000e-02,
         8.49000000e-01,  1.08000000e-01],
       [ 5.54621777e+01,  5.45200063e+01,  5.56268743e+01,
         1.33332000e+08,  5.49487038e+01,  4.33944089e-01,
         3.98266108e-02,  9.94000000e-01,  1.40000000e-02,
         8.48000000e-01,  1.38000000e-01],
       [ 5.47936893e+01,  5.35996280e+01,  5.50649597e+01,
         1.37160000e+08,  5.40355949e+01,  4.10279767e-01,
  

In [57]:
#Faktisk oppgang eller nedgang
y_test[:100]

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1])

In [58]:
#Sjekker om vi modellen predikerer rett
modelNB.predict(x_test[:100])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [59]:
#Sjekker sannsynligheten for oppgang og nedgang
Bayes = modelNB.predict_proba(x_test)

In [60]:
Bayes

array([[0.45173568, 0.54826432],
       [0.45081878, 0.54918122],
       [0.4524984 , 0.5475016 ],
       [0.4511127 , 0.5488873 ],
       [0.45097398, 0.54902602],
       [0.45067362, 0.54932638],
       [0.45062915, 0.54937085],
       [0.45080751, 0.54919249],
       [0.45185402, 0.54814598],
       [0.45069206, 0.54930794],
       [0.45134509, 0.54865491],
       [0.45069618, 0.54930382],
       [0.45142925, 0.54857075],
       [0.45258168, 0.54741832],
       [0.45274781, 0.54725219],
       [0.49201775, 0.50798225],
       [0.45240583, 0.54759417],
       [0.45347813, 0.54652187],
       [0.4536998 , 0.5463002 ],
       [0.45173512, 0.54826488],
       [0.45413239, 0.54586761],
       [0.45385385, 0.54614615],
       [0.45338953, 0.54661047],
       [0.45212783, 0.54787217],
       [0.45133693, 0.54866307],
       [0.45107459, 0.54892541],
       [0.45186155, 0.54813845],
       [0.45265591, 0.54734409],
       [0.45070923, 0.54929077],
       [0.45300559, 0.54699441],
       [0.

In [61]:
column_values = ["Nedgang", "Oppgang"]

df3 = pd.DataFrame(data = Bayes,
                  columns = column_values)

In [62]:
df3

,Nedgang,Oppgang
0,0.451736,0.548264
1,0.450819,0.549181
2,0.452498,0.547502
3,0.451113,0.548887
4,0.450974,0.549026
...,...,...
238,0.454424,0.545576
239,0.451316,0.548684
240,0.452938,0.547062
241,0.452997,0.547003


In [63]:
extracted_col = df3["Nedgang"]
extracted_col

0      0.451736
1      0.450819
2      0.452498
3      0.451113
4      0.450974
         ...   
238    0.454424
239    0.451316
240    0.452938
241    0.452997
242    0.457720
Name: Nedgang, Length: 243, dtype: float64

In [64]:
df1 = df1.join(extracted_col)
df1

,Open,Low,High,Volume,Close,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label,Nedgang
0,54.071930,53.866055,55.285368,195175200,54.503052,0.442600,0.154945,0.9403,0.034,0.873,0.094,1,0.451736
1,54.861504,54.737982,55.433104,173360400,55.132774,0.277222,0.079217,0.5106,0.056,0.877,0.066,1,0.450819
2,55.321696,54.890575,55.508190,109560400,55.309586,0.500270,0.190458,0.9844,0.043,0.849,0.108,0,0.452498
3,55.462178,54.520006,55.626874,133332000,54.948704,0.433944,0.039827,0.9940,0.014,0.848,0.138,0,0.451113
4,54.793689,53.599628,55.064960,137160000,54.035595,0.410280,0.135265,0.9840,0.035,0.856,0.109,0,0.450974
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,52.436624,51.836476,52.744076,89014800,52.257072,0.450048,0.184692,0.9945,0.055,0.794,0.151,0,0.454424
239,51.511804,49.438347,52.156227,187272000,49.841724,0.359338,0.062356,0.2350,0.056,0.881,0.063,0,0.451316
240,50.633720,50.436950,50.960850,104174400,50.788677,0.498261,0.176083,0.9979,0.034,0.808,0.159,1,0.452938
241,51.125642,50.060627,51.295356,103493200,50.215584,0.433536,0.170572,0.9349,0.046,0.850,0.104,0,0.452997


In [65]:
extracted_col1 = df3["Oppgang"]
extracted_col1

0      0.548264
1      0.549181
2      0.547502
3      0.548887
4      0.549026
         ...   
238    0.545576
239    0.548684
240    0.547062
241    0.547003
242    0.542280
Name: Oppgang, Length: 243, dtype: float64

In [66]:
df1 = df1.join(extracted_col1)
df1

,Open,Low,High,Volume,Close,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label,Nedgang,Oppgang
0,54.071930,53.866055,55.285368,195175200,54.503052,0.442600,0.154945,0.9403,0.034,0.873,0.094,1,0.451736,0.548264
1,54.861504,54.737982,55.433104,173360400,55.132774,0.277222,0.079217,0.5106,0.056,0.877,0.066,1,0.450819,0.549181
2,55.321696,54.890575,55.508190,109560400,55.309586,0.500270,0.190458,0.9844,0.043,0.849,0.108,0,0.452498,0.547502
3,55.462178,54.520006,55.626874,133332000,54.948704,0.433944,0.039827,0.9940,0.014,0.848,0.138,0,0.451113,0.548887
4,54.793689,53.599628,55.064960,137160000,54.035595,0.410280,0.135265,0.9840,0.035,0.856,0.109,0,0.450974,0.549026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,52.436624,51.836476,52.744076,89014800,52.257072,0.450048,0.184692,0.9945,0.055,0.794,0.151,0,0.454424,0.545576
239,51.511804,49.438347,52.156227,187272000,49.841724,0.359338,0.062356,0.2350,0.056,0.881,0.063,0,0.451316,0.548684
240,50.633720,50.436950,50.960850,104174400,50.788677,0.498261,0.176083,0.9979,0.034,0.808,0.159,1,0.452938,0.547062
241,51.125642,50.060627,51.295356,103493200,50.215584,0.433536,0.170572,0.9349,0.046,0.850,0.104,0,0.452997,0.547003


In [67]:
df1.head(60)

,Open,Low,High,Volume,Close,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label,Nedgang,Oppgang
0,54.071930,53.866055,55.285368,195175200,54.503052,0.442600,0.154945,0.9403,0.034,0.873,0.094,1,0.451736,0.548264
1,54.861504,54.737982,55.433104,173360400,55.132774,0.277222,0.079217,0.5106,0.056,0.877,0.066,1,0.450819,0.549181
2,55.321696,54.890575,55.508190,109560400,55.309586,0.500270,0.190458,0.9844,0.043,0.849,0.108,0,0.452498,0.547502
3,55.462178,54.520006,55.626874,133332000,54.948704,0.433944,0.039827,0.9940,0.014,0.848,0.138,0,0.451113,0.548887
4,54.793689,53.599628,55.064960,137160000,54.035595,0.410280,0.135265,0.9840,0.035,0.856,0.109,0,0.450974,0.549026
5,53.732841,53.456729,54.585395,150479200,53.599628,0.486064,0.068601,0.8146,0.068,0.840,0.091,0,0.450674,0.549326
6,53.514851,52.429781,53.732837,158066000,52.880280,0.517700,0.273559,0.9991,0.030,0.692,0.278,0,0.450629,0.549371
7,52.802776,52.451582,54.326238,142996000,54.217247,0.490909,0.327273,-0.2023,0.091,0.844,0.065,1,0.450808,0.549192
8,54.481249,53.246009,54.495780,197114800,53.543922,0.688636,0.296591,0.8402,0.000,0.875,0.125,0,0.451854,0.548146
9,54.137316,53.907224,55.307159,166825600,54.837284,0.376336,0.128820,0.9868,0.032,0.881,0.087,1,0.450692,0.549308


In [140]:
df1.to_csv("AppleOppgangNedgangUnik")

In [68]:
df1.sort_values("Oppgang", ascending = True)

,Open,Low,High,Volume,Close,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label,Nedgang,Oppgang
15,53.473684,52.628392,53.614162,384986800,52.718010,0.330929,0.119610,-0.7906,0.065,0.895,0.039,0,0.492018,0.507982
77,38.124593,36.367353,38.440557,382978400,36.634705,0.477962,0.107384,0.9721,0.049,0.840,0.111,0,0.491286,0.508714
45,50.753743,49.755862,51.746775,365314800,50.252380,0.502851,0.024342,0.9863,0.000,0.883,0.117,0,0.485133,0.514867
84,34.994134,34.512898,35.417039,365248800,34.559078,0.336233,0.206725,0.9779,0.037,0.788,0.175,0,0.485112,0.514888
222,53.029591,51.775034,54.242494,277125600,52.201385,0.450944,0.165832,1.0000,0.021,0.842,0.137,0,0.461990,0.538010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,50.253444,49.020784,50.629343,155054800,49.516289,0.463972,0.099725,0.9750,0.054,0.835,0.111,0,0.450636,0.549364
37,52.274770,52.001079,54.071919,155071200,53.945972,0.596656,0.216036,0.9950,0.039,0.785,0.177,1,0.450636,0.549364
133,45.120210,44.849269,45.725554,156171600,45.430202,0.432911,0.110788,-0.3818,0.061,0.878,0.062,1,0.450632,0.549368
63,43.819221,43.026884,43.828945,158126000,43.403610,0.502816,0.155111,0.8585,0.061,0.858,0.081,0,0.450629,0.549371


In [69]:
#Create and train the model (KNN)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 16)
knn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=16)

In [70]:
knn_score = knn.score(x_test, y_test)
print("Results for KNN Classifier:")
print(knn_score)

Results for KNN Classifier:
0.5185185185185185


In [71]:
y_pred_knn = knn.predict(x_test)

In [72]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm_knn = confusion_matrix(y_test, y_pred_knn)

In [73]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_knn).ravel()
print(tn, fp, fn, tp)

60 60 57 66


In [74]:
tpr_knn = round(tp/(tp + fn), 4)
tnr_knn = round(tn/(tn + fp), 4)

In [75]:
print(tpr_knn, tnr_knn)

0.5366 0.5


In [76]:
#Sjekker sannsynligheten for oppgang og nedgang
KNN = knn.predict_proba(x_test)

In [77]:
KNN

array([[0.3125, 0.6875],
       [0.4375, 0.5625],
       [0.5   , 0.5   ],
       [0.375 , 0.625 ],
       [0.25  , 0.75  ],
       [0.5625, 0.4375],
       [0.625 , 0.375 ],
       [0.625 , 0.375 ],
       [0.375 , 0.625 ],
       [0.6875, 0.3125],
       [0.3125, 0.6875],
       [0.75  , 0.25  ],
       [0.5   , 0.5   ],
       [0.4375, 0.5625],
       [0.1875, 0.8125],
       [0.625 , 0.375 ],
       [0.5625, 0.4375],
       [0.4375, 0.5625],
       [0.4375, 0.5625],
       [0.5625, 0.4375],
       [0.375 , 0.625 ],
       [0.5625, 0.4375],
       [0.4375, 0.5625],
       [0.1875, 0.8125],
       [0.3125, 0.6875],
       [0.3125, 0.6875],
       [0.4375, 0.5625],
       [0.375 , 0.625 ],
       [0.75  , 0.25  ],
       [0.5625, 0.4375],
       [0.375 , 0.625 ],
       [0.4375, 0.5625],
       [0.4375, 0.5625],
       [0.375 , 0.625 ],
       [0.5   , 0.5   ],
       [0.625 , 0.375 ],
       [0.3125, 0.6875],
       [0.5   , 0.5   ],
       [0.625 , 0.375 ],
       [0.5   , 0.5   ],


In [78]:
column_valuesApple = ["Nedgang-KNN", "Oppgang-KNN"]

df5 = pd.DataFrame(data = KNN,
                  columns = column_valuesApple)

In [79]:
df5

,Nedgang-KNN,Oppgang-KNN
0,0.3125,0.6875
1,0.4375,0.5625
2,0.5000,0.5000
3,0.3750,0.6250
4,0.2500,0.7500
...,...,...
238,0.4375,0.5625
239,0.5625,0.4375
240,0.6875,0.3125
241,0.6875,0.3125


In [80]:
extracted_colApple = df5["Nedgang-KNN"]
extracted_colApple

0      0.3125
1      0.4375
2      0.5000
3      0.3750
4      0.2500
        ...  
238    0.4375
239    0.5625
240    0.6875
241    0.6875
242    0.4375
Name: Nedgang-KNN, Length: 243, dtype: float64

In [81]:
df1 = df1.join(extracted_colApple)
df1

,Open,Low,High,Volume,Close,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label,Nedgang,Oppgang,Nedgang-KNN
0,54.071930,53.866055,55.285368,195175200,54.503052,0.442600,0.154945,0.9403,0.034,0.873,0.094,1,0.451736,0.548264,0.3125
1,54.861504,54.737982,55.433104,173360400,55.132774,0.277222,0.079217,0.5106,0.056,0.877,0.066,1,0.450819,0.549181,0.4375
2,55.321696,54.890575,55.508190,109560400,55.309586,0.500270,0.190458,0.9844,0.043,0.849,0.108,0,0.452498,0.547502,0.5000
3,55.462178,54.520006,55.626874,133332000,54.948704,0.433944,0.039827,0.9940,0.014,0.848,0.138,0,0.451113,0.548887,0.3750
4,54.793689,53.599628,55.064960,137160000,54.035595,0.410280,0.135265,0.9840,0.035,0.856,0.109,0,0.450974,0.549026,0.2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,52.436624,51.836476,52.744076,89014800,52.257072,0.450048,0.184692,0.9945,0.055,0.794,0.151,0,0.454424,0.545576,0.4375
239,51.511804,49.438347,52.156227,187272000,49.841724,0.359338,0.062356,0.2350,0.056,0.881,0.063,0,0.451316,0.548684,0.5625
240,50.633720,50.436950,50.960850,104174400,50.788677,0.498261,0.176083,0.9979,0.034,0.808,0.159,1,0.452938,0.547062,0.6875
241,51.125642,50.060627,51.295356,103493200,50.215584,0.433536,0.170572,0.9349,0.046,0.850,0.104,0,0.452997,0.547003,0.6875


In [82]:
extracted_col1Apple = df5["Oppgang-KNN"]
extracted_col1Apple

0      0.6875
1      0.5625
2      0.5000
3      0.6250
4      0.7500
        ...  
238    0.5625
239    0.4375
240    0.3125
241    0.3125
242    0.5625
Name: Oppgang-KNN, Length: 243, dtype: float64

In [83]:
df1 = df1.join(extracted_col1Apple)
df1

,Open,Low,High,Volume,Close,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label,Nedgang,Oppgang,Nedgang-KNN,Oppgang-KNN
0,54.071930,53.866055,55.285368,195175200,54.503052,0.442600,0.154945,0.9403,0.034,0.873,0.094,1,0.451736,0.548264,0.3125,0.6875
1,54.861504,54.737982,55.433104,173360400,55.132774,0.277222,0.079217,0.5106,0.056,0.877,0.066,1,0.450819,0.549181,0.4375,0.5625
2,55.321696,54.890575,55.508190,109560400,55.309586,0.500270,0.190458,0.9844,0.043,0.849,0.108,0,0.452498,0.547502,0.5000,0.5000
3,55.462178,54.520006,55.626874,133332000,54.948704,0.433944,0.039827,0.9940,0.014,0.848,0.138,0,0.451113,0.548887,0.3750,0.6250
4,54.793689,53.599628,55.064960,137160000,54.035595,0.410280,0.135265,0.9840,0.035,0.856,0.109,0,0.450974,0.549026,0.2500,0.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,52.436624,51.836476,52.744076,89014800,52.257072,0.450048,0.184692,0.9945,0.055,0.794,0.151,0,0.454424,0.545576,0.4375,0.5625
239,51.511804,49.438347,52.156227,187272000,49.841724,0.359338,0.062356,0.2350,0.056,0.881,0.063,0,0.451316,0.548684,0.5625,0.4375
240,50.633720,50.436950,50.960850,104174400,50.788677,0.498261,0.176083,0.9979,0.034,0.808,0.159,1,0.452938,0.547062,0.6875,0.3125
241,51.125642,50.060627,51.295356,103493200,50.215584,0.433536,0.170572,0.9349,0.046,0.850,0.104,0,0.452997,0.547003,0.6875,0.3125


In [84]:
df1.head(60)

,Open,Low,High,Volume,Close,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label,Nedgang,Oppgang,Nedgang-KNN,Oppgang-KNN
0,54.071930,53.866055,55.285368,195175200,54.503052,0.442600,0.154945,0.9403,0.034,0.873,0.094,1,0.451736,0.548264,0.3125,0.6875
1,54.861504,54.737982,55.433104,173360400,55.132774,0.277222,0.079217,0.5106,0.056,0.877,0.066,1,0.450819,0.549181,0.4375,0.5625
2,55.321696,54.890575,55.508190,109560400,55.309586,0.500270,0.190458,0.9844,0.043,0.849,0.108,0,0.452498,0.547502,0.5000,0.5000
3,55.462178,54.520006,55.626874,133332000,54.948704,0.433944,0.039827,0.9940,0.014,0.848,0.138,0,0.451113,0.548887,0.3750,0.6250
4,54.793689,53.599628,55.064960,137160000,54.035595,0.410280,0.135265,0.9840,0.035,0.856,0.109,0,0.450974,0.549026,0.2500,0.7500
5,53.732841,53.456729,54.585395,150479200,53.599628,0.486064,0.068601,0.8146,0.068,0.840,0.091,0,0.450674,0.549326,0.5625,0.4375
6,53.514851,52.429781,53.732837,158066000,52.880280,0.517700,0.273559,0.9991,0.030,0.692,0.278,0,0.450629,0.549371,0.6250,0.3750
7,52.802776,52.451582,54.326238,142996000,54.217247,0.490909,0.327273,-0.2023,0.091,0.844,0.065,1,0.450808,0.549192,0.6250,0.3750
8,54.481249,53.246009,54.495780,197114800,53.543922,0.688636,0.296591,0.8402,0.000,0.875,0.125,0,0.451854,0.548146,0.3750,0.6250
9,54.137316,53.907224,55.307159,166825600,54.837284,0.376336,0.128820,0.9868,0.032,0.881,0.087,1,0.450692,0.549308,0.6875,0.3125


In [130]:
df1.to_csv("AppleOppgangNedgangUnik-KNN")

In [85]:
#Create support vector machine
from sklearn import svm
svcl = svm.SVC()
svcl.fit(x_train, y_train)

SVC()

In [86]:
svcl_score = svcl.score(x_test, y_test)
print("Results for Support Machine Vector:")
print(svcl_score)

Results for Support Machine Vector:
0.5308641975308642


In [87]:
y_pred_sv = svcl.predict(x_test)

In [88]:
#Confusion Matrix
cm_sv = confusion_matrix(y_test, y_pred_sv)

In [89]:
tn1, fp1, fn1, tp1 = confusion_matrix(y_test, y_pred_sv).ravel()
print(tn1, fp1, fn1, tp1)

6 114 0 123


In [90]:
tpr_sv = round(tp1/(tp1 + fn1), 4)
tnr_sv = round(tn1/(tn1 + fp1), 4)

print(tpr_sv, tnr_sv)

1.0 0.05


In [91]:
svcl = probability=True

In [92]:
#Sjekker sannsynligheten for oppgang og nedgang
SVM = svcl.predict_proba(x_test)

AttributeError: 'bool' object has no attribute 'predict_proba'

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [96]:
X_train_vec = vectorizer.fit_transform(x_train)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'